In [ ]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = 'user'
password = 'grad5100user'
awsresource = 'database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com'
port = 3306

dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)


In [15]:
pip install pymysql

In [ ]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [16]:

chefdetails_query = """
SELECT season, seasonNumber, name, placement, gender
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6';
"""


chefdetails_df = pd.DataFrame(run_sql(chefdetails_query, engine))
chefdetails_df



,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female
...,...,...,...,...,...
317,World All Stars,20,May Phattanant Thongthong,14,Female
318,World All Stars,20,Begona Rodrigo,12,Female
319,World All Stars,20,Gabriel Rodriguez,2,Male
320,World All Stars,20,Amar Santana,6,Male


In [17]:
# Similar query for judges
judges_query = """
SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC
FROM judges
WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6';
"""

judges_df = pd.DataFrame(run_sql(judges_query, engine))
judges_df

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None
...,...,...,...,...,...,...
628,World All Stars,20,12,Quickfire,Sam Bompas,None
629,World All Stars,20,12,Elimination,Jeremy Chan,None
630,World All Stars,20,13,Quickfire,Greg Marchand,None
631,World All Stars,20,13,Elimination,Greg Marchand,None


In [18]:
season_query = "SELECT DISTINCT season, seasonNumber FROM chefdetails;"
season_df = pd.DataFrame(run_sql(season_query, engine))
season_df

,season,seasonNumber
0,All Stars: New York,8
1,All-Stars L.A.,17
2,Boston,12
3,California,13
4,Canada 6,6
5,Charleston,14
6,Chicago,4
7,D.C.,7
8,Colorado,15
9,Houston,19


In [20]:
gender_count = chefdetails_df['gender'].value_counts()
print(gender_count)

#152 Female

Male      170
Female    152
Name: gender, dtype: int64


In [21]:
winners_gender_count = chefdetails_df[chefdetails_df['placement'] == 1]['gender'].value_counts()
print(winners_gender_count)
#female 6

Male      14
Female     6
Name: gender, dtype: int64


In [22]:
top_3_gender_count = chefdetails_df[chefdetails_df['placement'].isin([1, 2, 3])]['gender'].value_counts()
print(top_3_gender_count)
#female 25

Male      36
Female    25
Name: gender, dtype: int64


In [23]:
female_top_3 = top_3_gender_count.get('Female', 0)
female_winners = winners_gender_count.get('Female', 0)

print("Number of times a female placed in the top 3:", female_top_3)
print("Number of times a female won:", female_winners)



Number of times a female placed in the top 3: 25
Number of times a female won: 6


Based on the result, females have placed in the top 3 a total of 25 times but have only won 6 of those times, resulting in a win rate of 24% among top 3 placements. This figure alone suggests that females are less likely to win when they reach the top 3 compared to their representation at this stage of the competition.

In [25]:

winners_df = chefdetails_df[chefdetails_df['placement'] == 1][['name', 'season']]

winners_df['guestJudgeStatus'] = winners_df['name'].isin(judges_df['guestJudge']).map({True: 'Yes', False: 'No'})

print(winners_df)


                     name               season guestJudgeStatus
0           Richard Blais  All Stars: New York              Yes
18           Melissa King       All-Stars L.A.              Yes
33                Mei Lin               Boston               No
49            Jeremy Ford           California               No
66      Brooke Williamson           Charleston              Yes
82        Stephanie Izard              Chicago              Yes
98           Kevin Sbraga                 D.C.               No
115          Joseph Flamm             Colorado               No
131            Kah-wai Lo              Houston               No
146  Kelsey Barnard Clark             Kentucky              Yes
162     Michael Voltaggio            Las Vegas              Yes
179             Ilan Hall          Los Angeles              Yes
194            Hung Huynh                Miami              Yes
209         Nicholas Elmi          New Orleans               No
228       Hosea Rosenberg             Ne

In [26]:

non_winners_df = chefdetails_df[chefdetails_df['placement'] != 1]


guest_judge_non_winners = non_winners_df[non_winners_df['name'].isin(judges_df['guestJudge'])]['name'].unique()

print(guest_judge_non_winners)


['Mike Isabella' 'Antonia Lofaso' 'Tiffany Derry' 'Dale Talde'
 'Spike Mendelsohn' 'Stephen Asprinio' 'Bryan Voltaggio' 'Gregory Gourdet'
 'Karen Akunowicz' 'Lee Anne Wong' 'Nini Nguyen' 'Melissa King'
 'Amar Santana' 'Kwame Onwuachi' 'Sheldon Simeon' 'Sam Talbot'
 'Richard Blais' 'Adrienne Cheatham' 'Carrie Baird' 'Josie Smith-Malave'
 'Nina Compton' 'Stefan Richter' 'Dawn Burrell' 'Shota Nakajima'
 'Miguel Morales' 'Brooke Williamson' 'Edward Lee' 'Nyesha Arrington']
